In [1]:
from auto_mm_bench.datasets import dataset_registry

# use these keys to specify which dataset to load
print(dataset_registry.list_keys())

train_dataset = dataset_registry.create("product_sentiment_machine_hack", "train")
test_dataset = dataset_registry.create("product_sentiment_machine_hack", "test")
print(train_dataset.data)

['product_sentiment_machine_hack', 'jigsaw_unintended_bias', 'jigsaw_unintended_bias100K', 'google_qa_label', 'google_qa_answer_helpful', 'google_qa_answer_plausible', 'google_qa_answer_type_procedure', 'google_qa_answer_type_reason_explanation', 'google_qa_question_type_reason_explanation', 'google_qa_answer_satisfaction', 'women_clothing_review', 'melbourne_airbnb', 'mercari_price_suggestion', 'ae_price_prediction', 'mercari_price_suggestion100K', 'imdb_genre_prediction', 'fake_job_postings', 'kick_starter_funding', 'jc_penney_products', 'wine_reviews', 'news_popularity', 'news_channel', 'news_popularity2', 'fake_job_postings2', 'bookprice_prediction', 'data_scientist_salary', 'california_house_price']


100%|██████████| 611k/611k [00:00<00:00, 1.62MiB/s]


100%|██████████| 154k/154k [00:00<00:00, 571kiB/s] 

      Unnamed: 0  Text_ID                                Product_Description  \
0           5743     2333  #techcrunch #google This Post Has Nothing to d...   
1           3042     3448  Data is the new oil. (Companies like Google an...   
2           4359      720  my sister is throwing the Google sxsw party to...   
3           5685     2328  Clear +succinct visions make for great UX (thi...   
4           2078     4955  40% of Google Maps use is mobile marissamayer ...   
...          ...      ...                                                ...   
5086        1096     3235  I'm eyeing the grilled cheese stand for after ...   
5087        3519     2728  Let's make that a temp-to-hire position RT @me...   
5088         212     1370  @mention -&gt; RT @mention New #UberSocial for...   
5089        2786     2530  @mention Did you find out the hours of the app...   
5090        3942     8162  &quot;At SXSW, Apple schools the marketing exp...   

      Product_Type  Sentiment  
0      

In [14]:
info_cols = [
    # 'splits',
    "feature_columns",
    "feature_types",
    "label_columns",
    "label_types",
    #  'data',
    "metric",
    "problem_type",
]

for col in info_cols:
    print(col, getattr(train_dataset, col))

feature_columns ['Product_Description', 'Product_Type']
feature_types ['text', 'categorical']
label_columns ['Sentiment']
label_types ['categorical']
metric acc
problem_type multiclass


In [ ]:
from datasets import Dataset
import pandas as pd

df = pd.DataFrame({"a": [1, 2, 3]})
dataset = Dataset.from_pandas(df)

In [16]:
train_dataset.label_columns[0]


'Sentiment'

In [23]:
from datasets import Dataset, DatasetDict

dataset_name = "product_sentiment_machine_hack"
cols = train_dataset.feature_columns + train_dataset.label_columns

train_txt = train_dataset.data[cols]
test_txt = test_dataset.data[cols]
train_dataset.data[train_dataset.label_columns[0]] = train_dataset.data[
    train_dataset.label_columns[0]
].astype(int)
test_dataset.data[train_dataset.label_columns[0]] = test_dataset.data[
    train_dataset.label_columns[0]
].astype(int)

# load dataset from dataframe
train_ds = Dataset.from_pandas(train_txt)
train_ds = train_ds.class_encode_column(train_dataset.label_columns[0])
test_ds = Dataset.from_pandas(test_txt)
test_ds = test_ds.class_encode_column(train_dataset.label_columns[0])


train_ds = train_ds.train_test_split(
    test_size=0.15, seed=42, stratify_by_column=train_dataset.label_columns[0]
)
ds = DatasetDict(
    {"train": train_ds["train"],
        "validation": train_ds["test"], "test": test_ds}
)
ds.push_to_hub(dataset_name)

Pushing split train to the Hub.                                         
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:02<00:00,  2.18s/it]
Pushing split validation to the Hub.
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.79s/it]
Pushing split test to the Hub.
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.82s/it]


In [22]:
ds

DatasetDict({
    train: Dataset({
        features: ['Product_Description', 'Product_Type', 'Sentiment'],
        num_rows: 4327
    })
    validation: Dataset({
        features: ['Product_Description', 'Product_Type', 'Sentiment'],
        num_rows: 764
    })
    test: Dataset({
        features: ['Product_Description', 'Product_Type', 'Sentiment'],
        num_rows: 1273
    })
})

In [25]:
test_dataset.data[test_dataset.label_columns[0]].value_counts()

2    748
3    436
1     75
0     14
Name: Sentiment, dtype: int64

In [26]:
from datasets import load_dataset

my_ds = load_dataset("james-burton/product_sentiment_machine_hack")


HfHubHTTPError: 502 Server Error: Bad Gateway for url: https://huggingface.co/api/datasets/james-burton/product_sentiment_machine_hack